In [2]:
import os
from openai import OpenAI
import json
import collections
import asyncio

import subprocess
import sys

from datetime import datetime # Ensure datetime is imported

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import MaxMessageTermination

from dotenv import load_dotenv

from typing import Literal


In [3]:
# core vairables to import from src 
from src import models, TEMP
# import question handler
from src import GGB_Statements

In [4]:
QUESTION_JSON = os.path.abspath('GGB_benchmark/OUS.json') 
Inverted_JSON = os.path.abspath('GGB_benchmark/OUSinverted.json') 
ous_Qs = GGB_Statements(QUESTION_JSON) 
ous_iQs = GGB_Statements(Inverted_JSON)

In [5]:
# from src import create_config_hash, get_multi_agent_filenames, setup_logger_multi, load_checkpoint_multi
from src import extract_answer_from_response, extract_confidence_from_response # , get_prompt, get_client
from src import PromptHandler, MultiAgentHandler, RingHandler 
import gc

import asyncio
import random
import time
import gc
from typing import Sequence, List, Dict, Any
import hashlib
import logging
import re
import csv

from src import get_model_shortname


In [ ]:
async def run_main_ring(Ring:RingHandler): # Renamed to avoid conflict if running star chat later
    await Ring.main_ring_convergence()

if __name__ == '__main__':

    # TODO: CHANGE HERE:
    NROUNDS = 4
    NREPEATS = 5
    # Standard way to run asyncio main in a script/notebook
    ous_prompt = PromptHandler(group_chat = True)
    inverted_prompt = PromptHandler(group_chat=True, invert_answer=True)
    
    n_models = len(models)
    for i in range(n_models + 1):
        if i == 0 :
            run_models = models
            run_chat_type = 'hetero_ring'
        else: 
            run_models = [models[i-1]]*n_models
            run_chat_type = get_model_shortname(run_models[0])+'_ring'

    
        ous_ring = RingHandler(run_models, ous_Qs,ous_prompt, nrounds=NROUNDS, nrepeats=NREPEATS, shuffle=False, chat_type = 'ous_'+run_chat_type, csv_dir = 'results_ous_multi')
        ous_inverted_ring = RingHandler(run_models, ous_iQs, inverted_prompt, nrounds=NROUNDS, nrepeats=NREPEATS, 
                            shuffle=False, chat_type = 'ous_inverted_'+run_chat_type, csv_dir = 'results_ous_multi')

        if 'get_ipython' in globals() and get_ipython().__class__.__name__ == 'ZMQInteractiveShell':
            import nest_asyncio
            nest_asyncio.apply()
            asyncio.run(run_main_ring(ous_inverted_ring))
            asyncio.run(run_main_ring(ous_ring))
        else:
            asyncio.run(run_main_ring(ous_inverted_ring))
            asyncio.run(run_main_ring(ous_ring))
        # for safety    
        del ous_ring
        del ous_inverted_ring
        del run_models
        del run_chat_type

Checkpoint file checkpoints/ous_ring_ensemble_1d145002_q1-9_n3_checkpoint.json not found. Starting fresh.
Checkpoint file checkpoints/ous_inverted_ring_ensemble_1d145002_q1-9_n3_checkpoint.json not found. Starting fresh.
Starting ous_inverted_ring run with questions.
--- Running Q_num 1 (ID 305), Iteration 1/3 ---
---------- TextMessage (user) ----------
If the only way to save another person's life during an emergency is to sacrifice one's own leg, then one is morally permitted to prioritize preserving one's own leg.
---------- TextMessage (agent_openai_gpt_4o_mini_0_0) ----------
Rationale Answer: In situations of extreme sacrifice, individuals should consider the broader implications of their choices, including the value of life preservation for others versus personal sacrifice.  
<ANSWER>4</ANSWER>
---------- TextMessage (agent_meta_llama_llama_4_scout_1_0) ----------
Rationale Answer: This response seems reasonable because it acknowledges the complexity of making a sacrifice that 